# RNN with tensorflow2.0

In [1]:
import tensorflow as tf
import numpy as np

## Be sure to used Tensorflow 2.0

In [2]:
assert hasattr(tf, "function") # Be sure to use tensorflow 2.0

## Open and process dataset

In [3]:
# You can used your own dataset with english text

with open("rnn_dataset/victorhugo.txt", "r") as f:
    text = f.read()

print(len(text))

print(text[:1000])


127286
Parce que, jargonnant vêpres, jeûne et vigile,
Exploitant Dieu qui rêve au fond du firmament,
Vous avez, au milieu du divin évangile,
Ouvert boutique effrontément ;

Parce que vous feriez prendre à Jésus la verge,
Cyniques brocanteurs sortis on ne sait d'où ;
Parce que vous allez vendant la sainte vierge
Dix sous avec miracle, et sans miracle un sou ;

Parce que vous contez d'effroyables sornettes
Qui font des temples saints trembler les vieux piliers ;
Parce que votre style éblouit les lunettes
Des duègnes et des marguilliers ;

Parce que la soutane est sous vos redingotes,
Parce que vous sentez la crasse et non l'œillet,
Parce que vous bâclez un journal de bigotes
Pensé par Escobar, écrit par Patouillet ;

Parce qu'en balayant leurs portes, les concierges
Poussent dans le ruisseau ce pamphlet méprisé ;
Parce que vous mêlez à la cire des cierges
Votre affreux suif vert-de-grisé ;

Parce qu'à vous tout seuls vous faites une espèce
Parce qu'enfin, blanchis dehors et noirs dedans,

## Remove character and create vocab
![](./images/rnn_vocab.png)

In [4]:
import unidecode

text = unidecode.unidecode(text)
text = text.lower()

text = text.replace("2", "")
text = text.replace("1", "")
text = text.replace("8", "")
text = text.replace("5", "")
text = text.replace(">", "")
text = text.replace("<", "")
text = text.replace("!", "")
text = text.replace("?", "")
text = text.replace("-", "")
text = text.replace("$", "")

text = text.strip()

vocab = set(text)
print(len(vocab), vocab)

print(text[:1000])


34 {'n', ':', '.', ',', ' ', 'v', "'", 'c', 'a', 'k', 'e', 'm', 'y', '\n', 'l', 'w', 'd', 'p', 'q', 'g', 'z', '"', 'x', 'u', 'j', 'f', 'i', 'r', ';', 't', 'b', 'h', 'o', 's'}
parce que, jargonnant vepres, jeune et vigile,
exploitant dieu qui reve au fond du firmament,
vous avez, au milieu du divin evangile,
ouvert boutique effrontement ;

parce que vous feriez prendre a jesus la verge,
cyniques brocanteurs sortis on ne sait d'ou ;
parce que vous allez vendant la sainte vierge
dix sous avec miracle, et sans miracle un sou ;

parce que vous contez d'effroyables sornettes
qui font des temples saints trembler les vieux piliers ;
parce que votre style eblouit les lunettes
des duegnes et des marguilliers ;

parce que la soutane est sous vos redingotes,
parce que vous sentez la crasse et non l'oeillet,
parce que vous baclez un journal de bigotes
pense par escobar, ecrit par patouillet ;

parce qu'en balayant leurs portes, les concierges
poussent dans le ruisseau ce pamphlet meprise ;
parce qu

## Map each letter to int

In [5]:
vocab_size = len(vocab)

vocab_to_int = {l:i for i,l in enumerate(vocab)}
int_to_vocab = {i:l for i,l in enumerate(vocab)}

print("vocab_to_int", vocab_to_int)
print()
print("int_to_vocab", int_to_vocab)

print("\nint for e:", vocab_to_int["e"])
int_for_e = vocab_to_int["e"]
print("letter for %s: %s" % (vocab_to_int["e"], int_to_vocab[int_for_e]))

vocab_to_int {'n': 0, ':': 1, '.': 2, ',': 3, ' ': 4, 'v': 5, "'": 6, 'c': 7, 'a': 8, 'k': 9, 'e': 10, 'm': 11, 'y': 12, '\n': 13, 'l': 14, 'w': 15, 'd': 16, 'p': 17, 'q': 18, 'g': 19, 'z': 20, '"': 21, 'x': 22, 'u': 23, 'j': 24, 'f': 25, 'i': 26, 'r': 27, ';': 28, 't': 29, 'b': 30, 'h': 31, 'o': 32, 's': 33}

int_to_vocab {0: 'n', 1: ':', 2: '.', 3: ',', 4: ' ', 5: 'v', 6: "'", 7: 'c', 8: 'a', 9: 'k', 10: 'e', 11: 'm', 12: 'y', 13: '\n', 14: 'l', 15: 'w', 16: 'd', 17: 'p', 18: 'q', 19: 'g', 20: 'z', 21: '"', 22: 'x', 23: 'u', 24: 'j', 25: 'f', 26: 'i', 27: 'r', 28: ';', 29: 't', 30: 'b', 31: 'h', 32: 'o', 33: 's'}

int for e: 10
letter for 10: e


In [6]:
encoded = [vocab_to_int[l] for l in text]
encoded_sentence = encoded[:100]

print(encoded_sentence)

[17, 8, 27, 7, 10, 4, 18, 23, 10, 3, 4, 24, 8, 27, 19, 32, 0, 0, 8, 0, 29, 4, 5, 10, 17, 27, 10, 33, 3, 4, 24, 10, 23, 0, 10, 4, 10, 29, 4, 5, 26, 19, 26, 14, 10, 3, 13, 10, 22, 17, 14, 32, 26, 29, 8, 0, 29, 4, 16, 26, 10, 23, 4, 18, 23, 26, 4, 27, 10, 5, 10, 4, 8, 23, 4, 25, 32, 0, 16, 4, 16, 23, 4, 25, 26, 27, 11, 8, 11, 10, 0, 29, 3, 13, 5, 32, 23, 33, 4, 8]


In [7]:
decoded_sentence = [int_to_vocab[i] for i in encoded_sentence]
print(decoded_sentence)

['p', 'a', 'r', 'c', 'e', ' ', 'q', 'u', 'e', ',', ' ', 'j', 'a', 'r', 'g', 'o', 'n', 'n', 'a', 'n', 't', ' ', 'v', 'e', 'p', 'r', 'e', 's', ',', ' ', 'j', 'e', 'u', 'n', 'e', ' ', 'e', 't', ' ', 'v', 'i', 'g', 'i', 'l', 'e', ',', '\n', 'e', 'x', 'p', 'l', 'o', 'i', 't', 'a', 'n', 't', ' ', 'd', 'i', 'e', 'u', ' ', 'q', 'u', 'i', ' ', 'r', 'e', 'v', 'e', ' ', 'a', 'u', ' ', 'f', 'o', 'n', 'd', ' ', 'd', 'u', ' ', 'f', 'i', 'r', 'm', 'a', 'm', 'e', 'n', 't', ',', '\n', 'v', 'o', 'u', 's', ' ', 'a']


In [8]:
decoded_sentence = "".join(decoded_sentence)
print(decoded_sentence)

parce que, jargonnant vepres, jeune et vigile,
exploitant dieu qui reve au fond du firmament,
vous a


## Sample of one batch
<img src="./images/rnn_letter.png" width="400px" ></img>

In [9]:
inputs, targets = encoded, encoded[1:]

print("Inputs", inputs[:10])
print("Targets", targets[:10])

Inputs [17, 8, 27, 7, 10, 4, 18, 23, 10, 3]
Targets [8, 27, 7, 10, 4, 18, 23, 10, 3, 4]


## Method used to generate batch in sequence order

In [10]:
def gen_batch(inputs, targets, seq_len, batch_size, noise=0):
    # Size of each chunk
    chuck_size = (len(inputs) -1)  // batch_size
    # Numbef of sequence per chunk
    sequences_per_chunk = chuck_size // seq_len

    for s in range(0, sequences_per_chunk):
        batch_inputs = np.zeros((batch_size, seq_len))
        batch_targets = np.zeros((batch_size, seq_len))
        for b in range(0, batch_size):
            fr = (b*chuck_size)+(s*seq_len)
            to = fr+seq_len
            batch_inputs[b] = inputs[fr:to]
            batch_targets[b] = inputs[fr+1:to+1]
            
            if noise > 0:
                noise_indices = np.random.choice(seq_len, noise)
                batch_inputs[b][noise_indices] = np.random.randint(0, vocab_size)
            
        yield batch_inputs, batch_targets

for batch_inputs, batch_targets in gen_batch(inputs, targets, 5, 32, noise=0):
    print(batch_inputs[0], batch_targets[0])
    break

for batch_inputs, batch_targets in gen_batch(inputs, targets, 5, 32, noise=3):
    print(batch_inputs[0], batch_targets[0])
    break

[17.  8. 27.  7. 10.] [ 8. 27.  7. 10.  4.]
[ 8.  8. 27.  7.  8.] [ 8. 27.  7. 10.  4.]


## Create your own layer

In [11]:
class OneHot(tf.keras.layers.Layer):
    def __init__(self, depth, **kwargs):
        super(OneHot, self).__init__(**kwargs)
        self.depth = depth

    def call(self, x, mask=None):
        return tf.one_hot(tf.cast(x, tf.int32), self.depth)

Test if the layer works well

In [12]:
class RnnModel(tf.keras.Model):

    def __init__(self, vocab_size):
        super(RnnModel, self).__init__()
        # Convolutions
        self.one_hot = OneHot(len(vocab))

    def call(self, inputs):
        output = self.one_hot(inputs)
        return output

batch_inputs, batch_targets = next(gen_batch(inputs, targets, 50, 32))

print(batch_inputs.shape)

model = RnnModel(len(vocab))
output = model.predict(batch_inputs)

print(output.shape)

#print(output)

print("Input letter is:", batch_inputs[0][0])
print("One hot representation of the letter", output[0][0])

#assert(output[int(batch_inputs[0][0])]==1)


(32, 50)
(32, 50, 34)
Input letter is: 17.0
One hot representation of the letter [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


# Set up the model

<img src="./images/architecture_rnn.png" width="400px" ></img>

In [13]:
vocab_size = len(vocab)

### Creat the layers

# Set the input of the model
tf_inputs = tf.keras.Input(shape=(None,), batch_size=64)
# Convert each value of the  input into a one encoding vector
one_hot = OneHot(len(vocab))(tf_inputs)
# Stack LSTM cells
rnn_layer1 = tf.keras.layers.LSTM(128, return_sequences=True, stateful=True)(one_hot)
rnn_layer2 = tf.keras.layers.LSTM(128, return_sequences=True, stateful=True)(rnn_layer1)
# Create the outputs of the model
hidden_layer = tf.keras.layers.Dense(128, activation="relu")(rnn_layer2)
outputs = tf.keras.layers.Dense(vocab_size, activation="softmax")(hidden_layer)

### Setup the model
model = tf.keras.Model(inputs=tf_inputs, outputs=outputs)

## Check if we can reset the RNN cells

In [14]:
# Star by resetting the cells of the RNN
model.reset_states()

# Get one batch
batch_inputs, batch_targets = next(gen_batch(inputs, targets, 50, 64))

# Make a first prediction
outputs = model.predict(batch_inputs)
first_prediction = outputs[0][0]

# Reset the states of the RNN states
model.reset_states()

# Make an other prediction to check the difference
outputs = model.predict(batch_inputs)
second_prediction = outputs[0][0]

# Check if both prediction are equal
assert(set(first_prediction)==set(second_prediction))

## Set the loss and objectives

In [15]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(lr=0.001)

## Set some metrics to track the progress of the training

In [16]:
# Loss
train_loss = tf.keras.metrics.Mean(name='train_loss')
# Accuracy
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

## Set the train method and the predict method in graph mode

In [17]:
@tf.function
def train_step(inputs, targets):
    with tf.GradientTape() as tape:
        # Make a prediction on all the batch
        predictions = model(inputs)
        # Get the error/loss on these predictions
        loss = loss_object(targets, predictions)
    # Compute the gradient which respect to the loss
    gradients = tape.gradient(loss, model.trainable_variables)
    # Change the weights of the model
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    # The metrics are accumulate over time. You don't need to average it yourself.
    train_loss(loss)
    train_accuracy(targets, predictions)

@tf.function
def predict(inputs):
    # Make a prediction on all the batch
    predictions = model(inputs)
    return predictions

# Train the model

In [ ]:
model.reset_states()

for epoch in range(4000):
    for batch_inputs, batch_targets in gen_batch(inputs, targets, 100, 64, noise=13):
        train_step(batch_inputs, batch_targets)
    template = '\r Epoch {}, Train Loss: {}, Train Accuracy: {}'
    print(template.format(epoch, train_loss.result(), train_accuracy.result()*100), end="")
    model.reset_states()

## Save the model

In [ ]:
import json
model.save("model_rnn.h5")

with open("model_rnn_vocab_to_int", "w") as f:
    f.write(json.dumps(vocab_to_int))
with open("model_rnn_int_to_vocab", "w") as f:
    f.write(json.dumps(int_to_vocab))

# Generate some text

In [20]:
import random

model.reset_states()

size_poetries = 300

poetries = np.zeros((64, size_poetries, 1))
sequences = np.zeros((64, 100))
for b in range(64):
    rd = np.random.randint(0, len(inputs) - 100)
    sequences[b] = inputs[rd:rd+100]

for i in range(size_poetries+1):
    if i > 0:
        poetries[:,i-1,:] = sequences
    softmax = predict(sequences)
    # Set the next sequences
    sequences = np.zeros((64, 1))
    for b in range(64):
        argsort = np.argsort(softmax[b][0])
        argsort = argsort[::-1]
        # Select one of the strongest 4 proposals
        sequences[b] = argsort[0]

for b in range(64):
    sentence = "".join([int_to_vocab[i[0]] for i in poetries[b]])
    print(sentence)
    print("\n=====================\n")
        

 soldats  mais be distoire ; et sur toute chose.
et la verite, moynt, le ruste range et vivant.
le sorvent des mains laimsieres ange leur plaine ;
le soir tout a fais qui donne a la tombe une cueux,
que c'est la lueure et crache a l'ocean maisee,
que de la rerre est ma prison est charmante 
et je m'



 souffle sons fuit de pourrir augon vertunt.
allem d'eux aux morts joyeux, seuls de maie et grande bourser vos demes de la nuit sur la partue ;
je n'ai jamais souffert qu'on osat y trace fais lainsi vous croyez l'esprit du poetre fe l'aire
des fleurs sont par plaind tout sous les degots de ce vere,


e la sainte vierre,
la nature, ainsi que la france au fond de l'ombre.

vii


essait pour tout un huit, n'estce pas de rire 
le soir tombe en pred,ant le vide et la cipiee,
il passene de toi, pour l'amere repouverte,
tout a ces juste et mains lien sont distincess
fait que nous nous connez dans l'omb


 est un pubeur et l'aige un revagt.
le pres de l'aut, ou dans ce timain ne semour,
qu'a not

In [19]:
import json

with open("model_rnn_vocab_to_int", "r") as f:
    vocab_to_int = json.loads(f.read())
with open("model_rnn_int_to_vocab", "r") as f:
    int_to_vocab = json.loads(f.read())
    int_to_vocab = {int(key):int_to_vocab[key] for key in int_to_vocab}

model.load_weights("model_rnn.h5")